<a href="https://colab.research.google.com/github/MateoRivera/network-dynamics/blob/main/functions_for_exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Handle matrices and vectors
import numpy as np

# Handle Graphs
import networkx as nx

# Plot
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

## Algebraic graph theory

In [105]:
def compute_P(W):
    # P = D^-1 @ W
    D = np.diag(W.sum(axis=1))
    return np.linalg.inv(D) @ W

def compute_L(W):
    # L = D - W
    D = np.diag(W.sum(axis=1))
    return D - W

def compute_pi(W, target_eigenvalue = 1, use_transition_matrix=True, decimal_precision=3):
    P = compute_P(W) if use_transition_matrix else W
    values, vectors = np.linalg.eig(P.T)
    target_eigenvalue = [i for i, eigenvalue in enumerate(values) if np.isclose(values[i], target_eigenvalue)]

    pi = vectors[:, target_eigenvalue].real
    for i in range(pi.shape[1]):
        pi[:, i] *= np.linalg.norm(pi[:, i])
        pi[:, i] /= np.sum(pi[:, i])

    round_vectorized = np.vectorize(lambda x: round(x, decimal_precision))
    pi = round_vectorized(pi)
    print("⚠️ Note: This result is an approximation due to potential numerical errors ⚠️")
    return pi

def compute_pi_bar(W, decimal_precision = 3):
    return compute_pi(compute_L(W), target_eigenvalue=0, use_transition_matrix=False, decimal_precision=decimal_precision)

## Averaging models with stubborn nodes

In [117]:
s = [1, 6]
r = filter(lambda x: x not in s, range(8))
list(r)

[0, 2, 3, 4, 5, 7]

In [136]:
len(np.array(np.array([1,2])).shape)

1

In [137]:
def compute_equilibria_with_stubborn(W, stubborn_id, u):
    u = np.array(u)
    if len(u.shape) == 1: # Is not a column vector
        u = u.reshape(-1,1)
    P = compute_P(W)
    regular_id = list(filter(lambda x: x not in stubborn_id, range(len(W))))
    Q = P[np.ix_(regular_id, regular_id)]
    E = P[np.ix_(regular_id, stubborn_id)]
    I = np.eye(len(Q))

    return np.linalg.inv((I-Q)) @ (E @ u)

In [106]:
W = np.array([
    [0, 0, 1, 0, 0, 0, 0, 0, 0],
    [1, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 1, 0, 1, 1, 0, 0],
    [0, 0, 0, 1, 1, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 1],
    [0, 0, 0, 0, 0, 0, 1, 0, 1],
    [0, 0, 0, 0, 0, 0, 0, 1, 0]
])

compute_pi(W)

⚠️ Note: This result is an approximation due to potential numerical errors ⚠️


array([[ 0.333,  0.   ],
       [ 0.333,  0.   ],
       [ 0.333,  0.   ],
       [-0.   ,  0.   ],
       [-0.   , -0.   ],
       [-0.   ,  0.   ],
       [-0.   ,  0.2  ],
       [-0.   ,  0.4  ],
       [-0.   ,  0.4  ]])

In [112]:
compute_pi_bar(W, 5)

⚠️ Note: This result is an approximation due to potential numerical errors ⚠️


array([[ 0.33333,  0.01351],
       [ 0.33333,  0.01351],
       [ 0.33333,  0.01351],
       [ 0.     , -0.     ],
       [ 0.     ,  0.     ],
       [ 0.     , -0.     ],
       [ 0.     ,  0.23987],
       [ 0.     ,  0.23987],
       [ 0.     ,  0.47973]])

In [138]:
compute_equilibria_with_stubborn(W, [0,8], np.array([[0], [1]]))

array([[0.        ],
       [0.        ],
       [0.        ],
       [0.36363636],
       [0.45454545],
       [1.        ],
       [1.        ]])

In [139]:
compute_equilibria_with_stubborn(W, [0,8], [0,1])

array([[0.        ],
       [0.        ],
       [0.        ],
       [0.36363636],
       [0.45454545],
       [1.        ],
       [1.        ]])

In [ ]:
array([[0.        ],
       [0.        ],
       [0.        ],
       [0.36363636],
       [0.45454545],
       [1.        ],
       [1.        ]])

In [ ]:
L = np.array([
    [1, -1, 0, 0, 0, 0, 0, 0, 0],
    [0, 1, -1, 0, 0, 0, 0, 0, 0],
    [-1, 0, 1, -1, -1, 0, 0, 0, 0],
    [0, 0, 0, 1, -1, -1, 0, 0, 0],
    [0, 0, 0, 0, 4, -1, 0, 0, 0],
    [0, 0, 0, 0, -1, 3, 0, 0, 0],
    [0, 0, 0, 0, -1, -1, 1, -1, 0],
    [0, 0, 0, 0, 0, 0, 0, 2, -1],
    [0, 0, 0, 0, 0, 0, -1, -1, 1]
])

In [ ]:
R = np.linalg.eig(L)
R

EigResult(eigenvalues=array([ 8.88365811e-17+0.j       ,  1.50000000e+00+0.8660254j,
        1.50000000e+00-0.8660254j,  1.00000000e+00+0.j       ,
        4.61803399e+00+0.j       , -1.19348975e-15+0.j       ,
        1.99999999e+00+0.j       ,  2.00000001e+00+0.j       ,
        2.38196601e+00+0.j       ]), eigenvectors=array([[ 5.77350269e-01+0.j , -2.88675135e-01-0.5j, -2.88675135e-01+0.5j,
        -7.07106781e-01+0.j , -1.52640134e-02+0.j , -2.29797071e-02+0.j ,
        -4.02190379e-16+0.j ,  4.04625701e-16+0.j , -2.66125765e-02+0.j ],
       [ 5.77350269e-01+0.j , -2.88675135e-01+0.5j, -2.88675135e-01-0.5j,
        -6.13217009e-16+0.j ,  5.52257192e-02+0.j , -2.29797071e-02+0.j ,
         4.10650549e-16+0.j , -4.12706946e-16+0.j ,  3.67776762e-02+0.j ],
       [ 5.77350269e-01+0.j ,  5.77350269e-01+0.j ,  5.77350269e-01-0.j ,
        -1.28208716e-16+0.j , -1.99808529e-01+0.j , -2.29797071e-02+0.j ,
        -4.05820295e-16+0.j ,  4.08137703e-16+0.j , -5.08254985e-02+0.j ],
       

In [ ]:
np.linalg.norm(R.eigenvectors[:, 5])

0.9999999999999999

In [ ]:
L.dot([1/3, 1/3, 1/3, 0,0,0,0,0,0])

array([0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
L = L.T

In [ ]:
L

array([[ 1,  0, -1,  0,  0,  0,  0,  0,  0],
       [-1,  1,  0,  0,  0,  0,  0,  0,  0],
       [ 0, -1,  1,  0,  0,  0,  0,  0,  0],
       [ 0,  0, -1,  1,  0,  0,  0,  0,  0],
       [ 0,  0, -1, -1,  4, -1, -1,  0,  0],
       [ 0,  0,  0, -1, -1,  3, -1,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  1,  0, -1],
       [ 0,  0,  0,  0,  0,  0, -1,  2, -1],
       [ 0,  0,  0,  0,  0,  0,  0, -1,  1]])

In [ ]:
W = np.array([
    [0, 0, 1, 0, 0, 0, 0, 0, 0],
    [1, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 1, 0, 1, 1, 0, 0],
    [0, 0, 0, 1, 1, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 1],
    [0, 0, 0, 0, 0, 0, 1, 0, 1],
    [0, 0, 0, 0, 0, 0, 0, 1, 0]
])

D = np.array([
    [1, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 1, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 4, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 3, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 2, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 1]
])

In [ ]:
R = np.linalg.eig((np.linalg.inv(D).dot(W)).T)
R.eigenvalues

array([-0.5       +0.8660254j, -0.5       -0.8660254j,
        1.        +0.j       ,  0.        +0.j       ,
        1.        +0.j       ,  0.28867513+0.j       ,
       -0.5       +0.5j      , -0.5       -0.5j      ,
       -0.28867513+0.j       ])

In [ ]:
R.eigenvectors[:, 2] * np.linalg.norm(R.eigenvectors[:, 2]) / np.sum(R.eigenvectors[:, 2] * np.linalg.norm(R.eigenvectors[:, 2]))

array([ 0.33333333-0.j,  0.33333333-0.j,  0.33333333-0.j, -0.        -0.j,
       -0.        -0.j, -0.        -0.j, -0.        -0.j, -0.        -0.j,
       -0.        -0.j])

In [ ]:
(R.eigenvectors[:, 4] * np.linalg.norm(R.eigenvectors[:, 4]) / np.sum(R.eigenvectors[:, 4] * np.linalg.norm(R.eigenvectors[:, 4]))).real

array([ 8.20136516e-05,  8.20136516e-05,  8.20136516e-05,  1.42047221e-18,
       -4.58924749e-18,  1.07287885e-16,  1.99950792e-01,  3.99901584e-01,
        3.99901584e-01])

In [ ]:
R.eigenvectors[:, 4].real

array([ 1.36723055e-04,  1.36723055e-04,  1.36723055e-04,  2.36803625e-18,
       -7.65062798e-18,  1.78857143e-16,  3.33333324e-01,  6.66666648e-01,
        6.66666648e-01])

In [ ]:
P = np.linalg.inv(D) @ W
P

array([[0.        , 0.        , 1.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.        , 1.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 1.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.25      , 0.25      , 0.        ,
        0.25      , 0.25      , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.33333333, 0.33333333,
        0.        , 0.33333333, 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 1.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.5       , 0.        , 0.5       ],


In [ ]:
stubborn_id = [0, 8]
regular_id = list(range(1, 8))
print(stubborn_id, regular_id)
Q = P[np.ix_(regular_id, regular_id)]
E = P[np.ix_(regular_id, stubborn_id)]
Q, E

[0, 8] [1, 2, 3, 4, 5, 6, 7]


(array([[0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        ],
        [1.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        ],
        [0.        , 1.        , 0.        , 0.        , 0.        ,
         0.        , 0.        ],
        [0.        , 0.25      , 0.25      , 0.        , 0.25      ,
         0.25      , 0.        ],
        [0.        , 0.        , 0.33333333, 0.33333333, 0.        ,
         0.33333333, 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 0.        ,
         0.5       , 0.        ]]),
 array([[1. , 0. ],
        [0. , 0. ],
        [0. , 0. ],
        [0. , 0. ],
        [0. , 0. ],
        [0. , 1. ],
        [0. , 0.5]]))

In [ ]:
I = np.eye(len(Q))
u = np.array([[0], [1]])

In [ ]:
np.linalg.inv((I-Q)) @ (E @ u)

array([[0.        ],
       [0.        ],
       [0.        ],
       [0.36363636],
       [0.45454545],
       [1.        ],
       [1.        ]])